### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.large-data :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojure.java.io :as io]
            [clojure.data.csv :as csv]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [10]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(load-large-dataset "../../data-sorted-cleaned/data-CRSP-sorted-cleaned.csv" "main")
(set-main "main")
(init-portfolio "1962-07-02" 10)

"1962-07-02"

In [6]:
(next-date)
(get (deref available-tics) "F")
;; (get-date)

{:reference {:ewretd "0.006906999999999999", :HSICCD "3711.0", :CUM-RET 6.821748764702937E-4, :CFACSHR "34.987499", :date "1962-07-05", :INIT-PRICE 79.625, :OPENPRC "", :SECSTAT "R", :SHROUT "20.794", :TICKER "F", :APRC 79.67933670599962, :COMNAM "FORD MOTOR CO DEL", :PRIMEXCH "N", :TRDSTAT "A", :FACSHR "", :HEXCD "1", :LOG-RET 6.821748764702937E-4, :RET "0.001572", :EXCHCD "1", :CFACPR "61.800346", :DLRET "", :PRC "79.625", :vwretd "0.006013", :FACPR "", :CUSIP "34537086", :NCUSIP "", :PERMCO "20750", :DIVAMT "", :PERMNO "25785", :SHRCD "11", :sprtrn "0.005665", :VOL "12600.0", :SICCD "3711"}}

In [9]:
(count (deref order-record))
;; (end-order)

8574

In [3]:
(order-lazy "PPL" 10 :print true)

{"PPL" {:date "1962-07-02", :expiration 3, :quantity 10, :remaining false, :leverage true, :print true, :direct true}}

In [11]:
(time (while 
    (< (compare (get-date) "1972-02") 0)
          (order-lazy "PPL" 10 :direct false)
          (next-day)))

"Elapsed time: 95699.627009 msecs"


nil

In [11]:
(time (while 
    (< (compare (curr-date) "2016-02") 0)(next-day)))

"Elapsed time: 916551.674745 msecs"


nil

### Initialise portfolio （Go back here everytime you want to restart.）

In [ ]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 100000);

In [ ]:
(count (deref tics-info))

### Write a strategy

The following code implements a trading strategy called Golden Rule:

MA 50 cross above the MA 200 (golden cross)

MA 200 cross below the MA 50 (death cross)

So in the codes, MA50 and MA200 are compared on a daily basis, if golden cross occurs, then you set a buy order; if death cross occurs, then you set a sell order first 



In [ ]:
(time (do (def MA50-vec-aapl [])
          (def MA200-vec-aapl [])
          (def MA50-vec-f [])
          (def MA200-vec-f [])
          (while (not= (get-date) "2016-12-29")
            (do
    ;; write your trading strategy here
              (def tics (deref available-tics)) ;20 ms
              (def MA50-vec-aapl (get-prev-n-days :PRC 50 "AAPL" MA50-vec-aapl (get (get tics "AAPL"):reference)))
              (def MA200-vec-aapl (get-prev-n-days :PRC 200 "AAPL" MA200-vec-aapl (get (get tics "AAPL") :reference)))
              (def MA50-vec-f (get-prev-n-days :PRC 50 "F" MA50-vec-f (get (get tics "F"):reference)))
              (def MA200-vec-f (get-prev-n-days :PRC 200 "F" MA200-vec-f (get (get tics "F") :reference)))
              (let [[MA50 MA200] [(moving-average :PRC MA50-vec-aapl) (moving-average :PRC MA200-vec-aapl)]]
                (if (> MA50 MA200)
                  (order "AAPL" 1 :reference (get (get tics "AAPL") :reference) :print false) 
                  (order "AAPL" 0 :remaining true :reference (get (get tics "AAPL") :reference))))
              (let [[MA50 MA200] [(moving-average :PRC MA50-vec-f) (moving-average :PRC MA200-vec-f)]]
                (if (> MA50 MA200)
                  (order "F" 1 :reference (get (get tics "F") :reference) :print false) 
                  (order "F" 0 :remaining true :reference (get (get tics "F") :reference))))
              ;(update-eval-report (get-date))
              (next-date)))))
(.close wrtr)

In [ ]:
(count (deref order-record))


### Check portfolio record

In [ ]:
;; view final portfolio
(view-portfolio)

In [ ]:
;; view portfolio value and return
(view-portfolio-record)

### Generate evaluation report

In [ ]:
(eval-report)

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)